simaliar to the first attempt but with cleaner code, more flexible layer creation

In [ ]:
import numpy as np
import math
print("start!")


In [ ]:
def create_layer(layer_size, dtype=np.float32, **kwargs):
    layer = np.zeros(layer_size, dtype=dtype)
    return(layer)

def create_layer_weights(prev_ly_size, curr_ly_size, init=True):
    if init:
        ly_weights = np.random.rand(prev_ly_size, curr_ly_size)
    else:
        ly_weights = np.zeros(shape=(prev_ly_size, curr_ly_size))
    return(ly_weights)

def create_biases(ly_size):
    biases = np.random.rand(ly_size)
    return(biases)

class n_network:
    def __init__(self, layers, biases, wieghts):
        self.layers = layers
        self.biases = biases
        self.wieghts = wieghts

def create_network(in_ly_size, out_ly_size, hidden_ly_num, hidden_ly_size):
    in_layer = create_layer(in_ly_size)
    hidden_lys = []
    for index in range(hidden_ly_num):
        hidden_lys.append(create_layer(hidden_ly_size))
    out_layer = create_layer(out_ly_size)
    network_lys = [in_layer, *hidden_lys, out_layer]

    biases =[]
    for layers in network_lys:
        biases.append(create_biases(np.size(layers)))

    weights = []
    for i in range(1, len(network_lys), 1):
        weights.append( create_layer_weights(np.size(network_lys[i-1]), np.size(network_lys[i]), init=True))

    network = n_network(network_lys, biases, weights)
    return(network)


def sigx(number):
    #sigmond function
    try:
        sig_x = ( 1 / (1+ math.exp(-number)))
        #this does not woprk for large negitive numbers 
        # due to rounding errors leading to devide by zero so we'll add error handling
    except OverflowError:
        sig_x = 0
    return(sig_x)

def forward(input_vals, net:n_network):
    #first layer
    net.layers[0] = input_vals

    for index in range(1, len(net.layers), 1):
        net.layers[index] = np.dot(net.layers[(index-1)], net.wieghts[index-1])
        sigVec = np.vectorize(sigx)
        net.layers[index] = sigVec(net.layers[index])
    return(net)



In [ ]:
network = create_network(3,3,1,3)


network = forward([0,1,2],network)
print(network.layers[:])
print(network.wieghts[:])

print("ouput:",network.layers[-1])


In [ ]:
ply = np.array([1,2,3,4,5])
cly = [0,0,0]
cwight = np.ones(shape=(np.size(ply), np.size(cly)))
cwight = np.array([
[0,1,2],
[3,4,5],
[1,1,1],
[0,0,0],
[5,5,5]
])
#cwight = np.random.rand(np.size(ply), np.size(cly))
nly = np.dot(ply, cwight)

print(ply)
print(cly)
print(cwight)
print(nly)

